# Feature Selection

This notebook is trying out different strategies for feature selection based on sklearn as well as a few neural approaches.

## Dependencies

In [ ]:
from collections import Counter

import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from flax import linen as nn
from flax import optim
from keras.layers import Dense
from keras.models import Model, Sequential
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.feature_selection import (
    RFE,
    RFECV,
    SelectFromModel,
    SelectKBest,
    SelectPercentile,
    VarianceThreshold,
    chi2,
    f_classif,
)
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import LinearSVC

## Preprocess Dataset

Download dataset to parent data directory.

In [ ]:
!if [ ! -f ../data/dorothea.zip ]; then wget -P ../data/ https://archive.ics.uci.edu/static/public/169/dorothea.zip && unzip ../data/dorothea.zip -d ../data/; fi

In [ ]:
class DataPreprocessor:
    def __init__(self, features_file, targets_file):
        self.features_file = features_file
        self.targets_file = targets_file
        self.df = None
    
    def _preprocess(self):
        data = []
        with open(self.features_file, 'r') as f:
            for line in f:
                active_features = line.strip().split()
                data.append(pd.Series({int(feature): 1 for feature in active_features}))
        features = pd.concat(data, axis=1).T.fillna(0).sort_index(axis=1)
        targets = pd.read_csv(self.targets_file, header=None, names=["target"])
        self.df = pd.concat([features, targets], axis=1)
    
    def __call__(self):
        self._preprocess()
        return self.df

In [ ]:
wrangler = DataPreprocessor('../data/DOROTHEA/dorothea_train.data', '../data/DOROTHEA/dorothea_train.labels')

In [ ]:
df = wrangler()

In [ ]:
df.head()

In [ ]:
X, y = df.drop('target', axis=1), df['target']

## Feature Data Types

In [ ]:
feature_number_unique_values = {column: X[column].unique() for column in X.columns}

In [ ]:
unique_values_by_columns = [value for _, value in feature_number_unique_values.items()]
tuples = [tuple(np.sort(arr)) for arr in unique_values_by_columns]

# Count the frequencies
frequency_table = Counter(tuples)

# Convert Counter to DataFrame
df = pd.DataFrame.from_records(list(frequency_table.items()), columns=['Array', 'Frequency'])

print(df)


So we see that each feature takes binary values 1 or 0, at least 1 of each.

Let's look at the target `y`.

In [ ]:
y.unique()

The drug discovery target `y` is also binary, taking values `-1` and `1`.

# Feature Selection Algorithms

## Filter Based Univariate Feature Selection

Selecting features using univariate statistical tests of teh relationship between each feature and the target variable.

### Only Feature Filtering

#### Variance Threshold

Let's see a histogram of the variance of each feature.

In [ ]:
variances = X.var()

plt.hist(variances, bins='auto', log=True)
plt.title('Histogram of Variances')
plt.xlabel('Variance')
plt.ylabel('Frequency')
plt.show()

In [ ]:
log_variances = np.log(variances + 1e-9)

plt.hist(log_variances, bins='auto', log=True)
plt.title('Histogram of Log Variances', )
plt.xlabel('Log Variance')
plt.ylabel('Frequency')
plt.show()

In [ ]:
plt.hist(variances, bins='auto', density=True, cumulative=True, histtype='step', alpha=0.8)
plt.title('CDF of Variances')
plt.xlabel('Log Variance')
plt.ylabel('Cumulative Probability')
plt.grid(True)

# Calculate the median of variances
median = np.median(variances)

# Plot the median as a dotted line
plt.axvline(median, color='b', linestyle='dotted', linewidth=2, label=f'Median Variance: {median:.2f}')

plt.legend()
plt.show()

In [ ]:
plt.boxplot(variances)
plt.title('Boxplot of Variances')
plt.ylabel('Variance')
plt.grid(True)
plt.show()

In [ ]:
plt.hist(log_variances, bins='auto', density=True, cumulative=True, histtype='step', alpha=0.8)
plt.title('CDF of Log Variances')
plt.xlabel('Log Variance')
plt.ylabel('Cumulative Probability')
plt.grid(True)

# Calculate the median of log_variances
median = np.median(log_variances)

# Plot the median as a dotted thick blue line
plt.axvline(median, color='b', linestyle='dotted', linewidth=2, label=f'Median Variance: {median:.2f}')

plt.legend()
plt.show()

In [ ]:
plt.boxplot(variances)
plt.title('Boxplot of Variances')
plt.ylabel('Variance')
plt.grid(True)
plt.show()

In [ ]:
variance_cutoff = 0.01
selector = VarianceThreshold(threshold=variance_cutoff)
selected_features = selector.fit_transform(X)

### 

### Model Based Filtering: Feature To Target

It may be useful to one hot encode the target y.

In [ ]:
y_one_hot = OneHotEncoder(sparse_output=False).fit_transform(y.to_numpy().reshape(-1, 1))

Classification based univariate feature selection using SelectBestK, SelectPercentile.

In [ ]:
X_new = SelectKBest(f_classif, k=2).fit_transform(X, y)
X_new.shape

In [ ]:
X_new = SelectKBest(chi2, k=2).fit_transform(X, y)
X_new.shape

In [ ]:
# commented out slow running block
# X_new = SelectKBest(mutual_info_classif, k=2).fit_transform(X, y)
# X_new.shape

In [ ]:
X_new = SelectPercentile(f_classif, percentile=0.01).fit_transform(X, y)
X_new.shape

In [ ]:
X_new = SelectPercentile(chi2, percentile=0.01).fit_transform(X, y)
X_new.shape

In [ ]:
# commented out slow running block
# X_new = SelectPercentile(mutual_info_classif, percentile=0.01).fit_transform(X, y)
# X_new.shape

### Recursive Feature Elimination (RFE)

RFE is the same as Seqential Feature Selection with backward elimination. It recursively removes the weakest feature (according to some model ranking - often coefficients listed in `coef_`). 

Performing RFE using Logistic Regression

In [ ]:
logreg = LogisticRegression(penalty="l2")
selector = RFE(estimator=logreg, n_features_to_select=88110, step=1)
selector = selector.fit(X, y)

# Print the mask of selected features
print(selector.support_)

Cross Validate the number of features to eliminate in RFE routine.

In [ ]:
min_features_to_select = 88110  # Minimum number of features to consider
clf = LogisticRegression()
cv = StratifiedKFold(5)

rfecv = RFECV(
    estimator=clf,
    step=1,
    cv=cv,
    scoring="accuracy",
    min_features_to_select=min_features_to_select,
    n_jobs=2,
)

rfecv.fit(X, y)

print(f"Optimal number of features: {rfecv.n_features_}")

RFE with Support vector classification.

In [ ]:
min_features_to_select = 88110  # Minimum number of features to consider
clf = RandomForestClassifier()
cv = StratifiedKFold(5)

rfecv = RFECV(
    estimator=clf,
    step=1,
    cv=cv,
    scoring="accuracy",
    min_features_to_select=min_features_to_select,
    n_jobs=2,
)
rfecv.fit(X, y)

print(f"Optimal number of features: {rfecv.n_features_}")

### SelectFromModel

#### L1 based 

In [ ]:

lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X, y)
model = SelectFromModel(lsvc, prefit=True)
X_new = model.transform(X)
X_new.shape

In [ ]:
logistic_l1 = LogisticRegression(penalty='l1', solver='liblinear').fit(X, y)
model = SelectFromModel(logistic_l1, prefit=True)
X_new = model.transform(X)
X_new.shape

#### Tree based

In [ ]:
clf = ExtraTreesClassifier(n_estimators=50)
clf = clf.fit(X, y)
model = SelectFromModel(clf, prefit=True)
X_new = model.transform(X)
X_new.shape               

### Sequential Feature Selection

In [ ]:
# long running process
# knn = KNeighborsClassifier(n_neighbors=5)
# sfs_backward = SequentialFeatureSelector(
#     knn, n_features_to_select = X.shape[1] - 5, direction="backward"
# ).fit(X, y)

# print(
#     "Number Features selected by forward sequential selection: "
#     f"{len([sfs_forward.get_support()])}"
# )

### Pipeline Feature Selection

In [ ]:
clf = Pipeline([
  ('feature_selection', SelectFromModel(LinearSVC(dual="auto", penalty="l1"))),
  ('classification', RandomForestClassifier())
])
clf.fit(X, y)

### Feature Permutation Importance Based Selection

In [ ]:
# Custom transformer for permutation importance
class PermutationImportanceTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, model, scoring, n_repeats=10, random_state=0, k=10):
        self.model = model
        self.scoring = scoring
        self.n_repeats = n_repeats
        self.random_state = random_state
        self.k = k

    def fit(self, X, y):
        self.model.fit(X, y)
        self.importance = permutation_importance(self.model, X, y, scoring=self.scoring, n_repeats=self.n_repeats, random_state=self.random_state)
        self.selector = SelectKBest(k=self.k).fit(X, self.importance.importances_mean)
        return self

    def transform(self, X):
        return self.selector.transform(X)

# Assuming X and y are your data
# Define the pipeline
pipeline = Pipeline([
    ('feature_selection', PermutationImportanceTransformer(LogisticRegression(penalty='l2', solver='liblinear'), scoring='accuracy', k=10)),
    ('classification', LogisticRegression(penalty='l2', solver='liblinear'))
])

# Fit and use the pipeline
pipeline.fit(X, y)

### Neural Learning

In [ ]:
# Define the number of features
X = X.to_numpy()
n_features = X.shape[1]

# Define the encoder dimension
encoder_dim = 10  # change this to your desired number of features

# Define the autoencoder model
autoencoder = Sequential([
    Dense(encoder_dim, input_shape=(n_features,), activation='relu'),
    Dense(n_features, activation='sigmoid')
])

# Compile the autoencoder model
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Fit the autoencoder model
autoencoder.fit(X, X, epochs=50, batch_size=256, shuffle=True, validation_split=0.2)

# Define the encoder model
encoder = Model(inputs=autoencoder.input, outputs=autoencoder.layers[0].output)

# Transform the data
X_transformed = encoder.predict(X)

Done in Keras above and PyTorch below.

In [ ]:
# Assuming X is your data
# Convert X to a PyTorch tensor
X_tensor = torch.from_numpy(X.values).float()

# Define the number of features
n_features = X.shape[1]

# Define the encoder dimension
encoder_dim = 10  # change this to your desired number of features

# Define the autoencoder model
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(n_features, encoder_dim),
            nn.ReLU(True)
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoder_dim, n_features),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

autoencoder = Autoencoder()

# Define a loss function and an optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)

# Train the autoencoder
for epoch in range(50):
    autoencoder.train()
    optimizer.zero_grad()
    outputs = autoencoder(X_tensor)
    loss = criterion(outputs, X_tensor)
    loss.backward()
    optimizer.step()

# Use the encoder to transform the data
autoencoder.eval()
X_transformed = autoencoder.encoder(X_tensor).detach().numpy()

#### AutoEncoder

In [ ]:
# Assuming X is your data
# Define the number of features
n_features = X.shape[1]

# Define the encoder dimension
encoder_dim = 10  # change this to your desired number of features

# Define the autoencoder model
autoencoder = Sequential([
    Dense(encoder_dim, input_shape=(n_features,), activation='relu'),
    Dense(n_features, activation='sigmoid')
])

# Compile the autoencoder model
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Fit the autoencoder model
autoencoder.fit(X, X, epochs=50, batch_size=256, shuffle=True, validation_split=0.2)

# Define the encoder model
encoder = Model(inputs=autoencoder.input, outputs=autoencoder.layers[0].output)

# Transform the data
X_transformed = encoder.predict(X)

#### Regularization of neural network

In [ ]:

X = jnp.array(X.values)
y = jnp.array(y.values)

class Model(nn.Module):
    def setup(self):
        self.dense = nn.Dense(features=1)

    def __call__(self, x):
        x = self.dense(x)
        return nn.sigmoid(x)

def loss_fn(params, model, x, y, l1_reg):
    y_pred = model.apply(params, x)
    loss = jnp.mean(jnp.square(y_pred - y))
    l1_penalty = l1_reg * jnp.sum(jnp.abs(params))
    return loss + l1_penalty

# Define an optimizer
optimizer = optim.Adam(learning_rate=0.001)

# Initialize the model and optimizer
model = Model()
params = model.init(jax.random.PRNGKey(0), X)
opt_state = optimizer.create(params)

# Define a training step
def train_step(opt_state, x, y, l1_reg):
    params = optimizer.target(opt_state)
    loss, grads = jax.value_and_grad(loss_fn)(params, model, x, y, l1_reg)
    opt_state = optimizer.update(grads, opt_state)
    return opt_state, loss

# Train the model
l1_reg = 0.01  # adjust this to your desired level of regularization
for _ in range(1000):
    opt_state, loss = train_step(opt_state, X, y, l1_reg)

# Use the model to make predictions
params = optimizer.target(opt_state)
y_pred = model.apply(params, X)

#### TabTransformer based selection

In [ ]:
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Assuming X and y are your data
# Preprocess the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert categorical features to integers
for col in X_train.columns[X_train.dtypes == 'object']:
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col])
    X_test[col] = le.transform(X_test[col])

# Define the TabNet model
clf = TabNetClassifier()

# Fit the model
clf.fit(
  X_train.values, y_train.values,
  eval_set=[(X_train.values, y_train.values), (X_test.values, y_test.values)],
  eval_name=['train', 'valid'],
  eval_metric=['auc'],
  max_epochs=1000 , patience=20,
  batch_size=256, virtual_batch_size=128,
  num_workers=0,
  weights=1,
  drop_last=False
)

# Make predictions
preds = clf.predict(X_test.values)